This code is to extract the emitters and the isins of the green bonds to ultimately get a data set of all green bond emitting companies which we can use to build our conventional bond data. With the help of the ISIN we will be able to get external credit rating data if needed. The URL's of this matched data will allow us to web scrape all the relevant spread data at the end.

# Code for all bond links
The code below is for the file "raw_all_bond_links.csv"

In [ ]:
import pandas as pd
import re
import os

# Loading the data
data_filepath = os.path.join("..", "data", "Static_data", "raw_all_bond_links.csv")
df = pd.read_csv(data_filepath)
print(df.head())
df.columns = ['URL'] # Set column name to URL as per requirement

# Function to extract ISIN and Company + Cupon and Maturity
def extract_isin_company(url):
    parts = url.split('/')
    bond_info = parts[-1] # Get the last part after "anleihe/"
    if '-' in bond_info:
        isin_company_parts = bond_info.split('-', 1) # Split only once at the first hyphen
        isin = isin_company_parts[0]
        company_coupon_maturity = isin_company_parts[1]
        return isin, company_coupon_maturity
    else:
        return None, bond_info # Handle cases without hyphen separation if needed

# Apply the extraction function
df[['ISIN', 'Company + Kupon and Maturity']] = df['URL'].apply(extract_isin_company).tolist()

# Define the function to split coupon and maturity (as provided by the user)
def split_coupon_maturity(text):
    match = re.search(r"(\d+(?:[.,]\d+)?(?:-\d+)+)$", text)  # Corrected regex for decimal and comma
    if match:
        coupon_maturity = match.group(1)
        company = text[:match.start()].rstrip('-')
        return company, coupon_maturity
    return text.rstrip('-'), None

# Apply the function to create 'Company' and 'Kupon_Maturity' columns
df[['Company', 'Kupon_Maturity']] = df['Company + Kupon and Maturity'].apply(split_coupon_maturity).tolist()

df['Company'] = df['Company'].str.replace(r'^-', '', regex=True)

# Define the function to split coupon, maturity start and end (as provided by the user)
def split_coupon_maturity_details(text):
    if text:
        parts = text.split('-')
        if len(parts) >= 3:
            maturity_start = parts[-2]
            maturity_end = parts[-1]
            coupon = '-'.join(parts[:-2]).replace('-', '.')
            return coupon, maturity_start, maturity_end
        elif len(parts) == 2: # Handle cases with only coupon and end year, assuming start year is the same
            maturity_end = parts[-1]
            coupon = parts[0].replace('-', '.')
            return coupon, None, maturity_end # Maturity start is none if not present
    return None, None, None

# Apply the function to create 'Coupon', 'Maturity_Start', and 'Maturity_End' columns
df[['Coupon', 'Maturity_Start', 'Maturity_End']] = df['Kupon_Maturity'].apply(split_coupon_maturity_details).tolist()

# To ensure all columns with numbers are numbers
columns_to_convert = ['Coupon', 'Maturity_Start', 'Maturity_End']
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Save the updated DataFrame to a CSV file
df.to_csv("All_Bond_URLs_Processed.csv", index=False)

print("Data processed and saved to raw_all_bond_links.csv")

# Display the DataFrame with the separated columns, including the original
print(df[['URL', 'ISIN', 'Company + Kupon and Maturity', 'Company', 'Coupon', 'Maturity_Start', 'Maturity_End']].head())

# Display the DataFrame with the separated columns (optional) as requested in the prompt
print(df[['Company', 'Kupon_Maturity']].head())

                                                   0
0  https://www.boerse-frankfurt.de/anleihe/de0001...
1  https://www.boerse-frankfurt.de/anleihe/no0010...
2  https://www.boerse-frankfurt.de/anleihe/de0001...
3  https://www.boerse-frankfurt.de/anleihe/de000b...
4  https://www.boerse-frankfurt.de/anleihe/us9128...
Data processed and saved to raw_all_bond_links.csv
                                                 URL          ISIN  \
0  https://www.boerse-frankfurt.de/anleihe/de0001...  de0001030716   
1  https://www.boerse-frankfurt.de/anleihe/no0010...  no0010757925   
2  https://www.boerse-frankfurt.de/anleihe/de0001...  de0001102382   
3  https://www.boerse-frankfurt.de/anleihe/de000b...  de000bu22023   
4  https://www.boerse-frankfurt.de/anleihe/us9128...  us91282ckb62   

             Company + Kupon and Maturity                     Company  Coupon  \
0  deutschland-bundesrepublik-0-000-20-25  deutschland-bundesrepublik   0.000   
1          norwegen-koenigreich-1-5-16-26       

# Code for Green bond links
The following code is slightly different as I already pre modified the green bond link data. This manual modification just created what the code above did in a first step, extracting the ISIN and the rest of the string into seperate columns, to subsequently run the below code that does a regex to extract the name of the bond emitters

In [ ]:
import pandas as pd
import re
import os

# Loading the data
data_filepath = os.path.join("..", "data", "Static_data", "Green_Bond_URLs.xlsx")
excel_file = pd.ExcelFile(data_filepath)
df = excel_file.parse("Data")  # Parse the "Data" sheet into a DataFrame
print(df.head())

def split_coupon_maturity(text):
    match = re.search(r"(\d+(?:\.\d+)?(?:-\d+)+)$", text)  # Improved regex
    if match:
        coupon_maturity = match.group(1)
        company = text[:match.start()].rstrip('-')  # Exclude trailing hyphens
        return company, coupon_maturity
    return text.rstrip('-'), None  # Handle cases where no match is found

df[['Company', 'Kupon_Maturity']] = df['Company + Kupon and Maturity'].apply(split_coupon_maturity).tolist()

df['Company'] = df['Company'].str.replace(r'^-', '', regex=True)

# Split Kupon_Maturity into Coupon, Maturity_Start, and Maturity_End
def split_coupon_maturity_details(text):
    if text:
        parts = text.split('-')
        if len(parts) >= 3:  # Ensure there are at least 3 parts (coupon and maturities)
            maturity_start = parts[-2]
            maturity_end = parts[-1]
            coupon = '-'.join(parts[:-2]).replace('-', '.')  # Join remaining parts for coupon
            return coupon, maturity_start, maturity_end
    return None, None, None  # Handle cases where splitting fails

df[['Coupon', 'Maturity_Start', 'Maturity_End']] = df['Kupon_Maturity'].apply(split_coupon_maturity_details).tolist()

# Save the updated DataFrame back to a new CSV file (or overwrite the original)
df.to_csv("Green_Bond_URLs_Processed.csv", index=False)  # Save to a new CSV

print("Data processed and saved to Green_Bond_URLs_Processed.csv")

# Display the DataFrame with the separated columns, including the original
print(df[['Company + Kupon and Maturity', 'Company', 'Coupon', 'Maturity_Start', 'Maturity_End']].head())

# Save the updated DataFrame back to a new CSV file (or overwrite the original)
df.to_csv("Green_Bond_URLs_Processed.csv", index=False)  # Save to a new CSV

print("Data processed and saved to Green_Bond_URLs_Processed.csv")

# Display the DataFrame with the separated columns (optional)
print(df[['Company', 'Kupon_Maturity']].head())

Data processed and saved to Green_Bond_URLs_Processed.csv
             Company + Kupon and Maturity                     Company Coupon  \
0  deutschland-bundesrepublik-0-000-20-25  deutschland-bundesrepublik  0.000   
1          norwegen-koenigreich-1-5-16-26        norwegen-koenigreich    1.5   
2      deutschland-bundesrepublik-1-15-25  deutschland-bundesrepublik      1   
3    deutschland-bundesrepublik-3-1-23-25  deutschland-bundesrepublik    3.1   
4    united-states-of-america-4-625-24-26    united-states-of-america  4.625   

  Maturity_Start Maturity_End  
0             20           25  
1             16           26  
2             15           25  
3             23           25  
4             24           26  
Data processed and saved to Green_Bond_URLs_Processed.csv
                      Company Kupon_Maturity
0  deutschland-bundesrepublik    0-000-20-25
1        norwegen-koenigreich      1-5-16-26
2  deutschland-bundesrepublik        1-15-25
3  deutschland-bundesrepublik  

# Code To Merge csv Files and Create The Bond Dictionary

In [ ]:
import pandas as pd
import os

def add_rows_based_on_company(all_bonds_df, green_bonds_df):
    try:
        # Get the set of unique company names from Green_Bond_URLs_Processed
        green_companies = set(green_bonds_df['Company'].str.strip()) # Using 'Company' as in the image and trimming whitespace

        rows_to_append = []

        # Iterate through each row of All_Bond_URLs_Processed
        for index, row in all_bonds_df.iterrows():
            company_name = str(row['Company']).strip() # Using 'Company' as in the image and trimming whitespace
            if company_name in green_companies:
                rows_to_append.append(row)

        # If there are rows to append, convert them to DataFrame and append to green_bonds_df
        if rows_to_append:
            append_df = pd.DataFrame(rows_to_append)
            green_bonds_df = pd.concat([green_bonds_df, append_df], ignore_index=True)

        # Save the updated Green_Bond_URLs_Processed.csv
        output_filepath = os.path.join("..", "data", "Static_data", "bond_dictionary") + "/bond_dictionary.csv"
        green_bonds_df.to_csv(output_filepath, index=False)
        print(green_bonds_df.info)
        print(f"Successfully added rows.")
        print(f"file saved to {output_filepath}")

    except FileNotFoundError:
        print("Error: One or both of the CSV files were not found. Please check the file paths.")
    except KeyError as e:
        print(f"Error: Column '{e}' not found in one or both of the CSV files. Please check the column names.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Loading the data
data_filepath = os.path.join("..", "data", "Static_data", "All_Bond_URLs_Processed.csv")
df_all = pd.read_csv(data_filepath)
print(df_all.head())

data_filepath_green = os.path.join("..", "data", "Static_data", "Green_Bond_URLs_Processed.csv")
df_green = pd.read_csv(data_filepath_green)
print(df_green.head())

add_rows_based_on_company(df_all, df_green)

                                                 URL          ISIN  \
0  https://www.boerse-frankfurt.de/anleihe/de0001...  de0001030716   
1  https://www.boerse-frankfurt.de/anleihe/no0010...  no0010757925   
2  https://www.boerse-frankfurt.de/anleihe/de0001...  de0001102382   
3  https://www.boerse-frankfurt.de/anleihe/de000b...  de000bu22023   
4  https://www.boerse-frankfurt.de/anleihe/us9128...  us91282ckb62   

             Company + Kupon and Maturity                     Company  \
0  deutschland-bundesrepublik-0-000-20-25  deutschland-bundesrepublik   
1          norwegen-koenigreich-1-5-16-26        norwegen-koenigreich   
2      deutschland-bundesrepublik-1-15-25  deutschland-bundesrepublik   
3    deutschland-bundesrepublik-3-1-23-25  deutschland-bundesrepublik   
4    united-states-of-america-4-625-24-26    united-states-of-america   

  Kupon_Maturity  Coupon  Maturity_Start  Maturity_End  
0    0-000-20-25   0.000            20.0          25.0  
1      1-5-16-26   1.500  

We have 9673 bonds in our merged data set. We have 147 Unique values, the same amount as in our original green bond data indicating that the merged on company name worked well.

# Code to merge bond_dictionary with Static_data

In [5]:
import pandas as pd

url_static = "https://github.com/Fijiman001/EGR-Empirical-Project/raw/main/data/Static_data/Cleaned_Static_Data.xlsx"
url_dictionary = "https://raw.githubusercontent.com/Fijiman001/EGR-Empirical-Project/refs/heads/main/data/Static_data/bond_dictionary/bond_dictionary_cleaned.csv"

df_bond_dictionary = pd.read_csv(url_dictionary)
df_static_data = pd.read_excel(url_static)
# print(df_bond_dictionary.head())
# print(df_static_data.head())

In [7]:
# Merging the two dataframes using a left join on the 'URL' column
df = df_bond_dictionary.merge(df_static_data, on='URL', how='left')

# Display the first few rows of the merged dataframe
print(df.head())
df.to_csv("bond_universe_raw.csv", index=False)

                                                 URL          ISIN  \
0  https://www.boerse-frankfurt.de/anleihe/xs2694...  xs2694874533   
1  https://www.boerse-frankfurt.de/anleihe/xs2694...  xs2694874533   
2  https://www.boerse-frankfurt.de/anleihe/xs2482...  xs2482887879   
3  https://www.boerse-frankfurt.de/anleihe/xs2482...  xs2482887879   
4  https://www.boerse-frankfurt.de/anleihe/de000a...  de000a3lh6t7   

  Company + Kupon and Maturity                                  Company  \
0                       #NAME?                  volkswagen-leasing-gmbh   
1                       #NAME?                  volkswagen-leasing-gmbh   
2                       #NAME?                                   rwe-ag   
3                       #NAME?                                   rwe-ag   
4                       #NAME?  mercedes-benz-international-finance-b-v   

  Kupon_Maturity  Coupon  Maturity_Start  Maturity_End  is_green  \
0     4-75-23-31    4.75              23            31      

# Code to Clean bond_universe

- We remove bonds that have "non passive" options: we remove entries in the column "SchuldnerkÃƒÂ¼ndigungsart" that have "call option".
- Remove duplicate column "Company + Kupon and Maturity"
- Remove any columns with missing data
- Remove duplicates based on columns "URL","ISIN","Company","Kupon_Maturity","Coupon","Maturity_Start","Maturity_End","is_green"
- Create emission year fixed effects, column "Emissionsdatum", value format is "25,09,2023"

We try removing entries with "special call"?
Remove entries where column "Branche" has entry "Schuldverschreibungen von Sonderinstituten"

In [89]:
df = pd.read_csv("bond_universe_raw.csv")

In [90]:
df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
df.columns = df.columns.str.replace("Ã¤", "ä").str.replace("Ã¼", "ü")  # Fix encoding issues

# 1. Remove bonds with a "Call option" in "Schuldnerkündigungsart"
df.drop(df[df['Schuldnerkündigungsart'] == 'Call option'].index, inplace=True)

# 2. Remove duplicate column "Company + Kupon and Maturity"
if "Company + Kupon and Maturity" in df.columns:
    df = df.drop(columns=["Company + Kupon and Maturity"])

# 4. Remove duplicates based on the selected columns while keeping all columns
subset_cols = [
    'URL', 'ISIN', 'Company', 'Kupon_Maturity', 'Coupon', 'Maturity_Start',
    'Maturity_End', 'is_green', 'Letzter Preis', 'Modified Duration', 'Kupon',
    'Emittent', 'Branche', 'Fälligkeit', 'Schuldnerkündigungsart', 'Sonderkündigung',
    'Nachrangig', 'Kleinste handelbare Einheit', 'Spezialist', 'Emissionsdatum',
    'Emissionsvolumen', 'Umlaufendes Volumen', 'Emissionswährung', 'Depotwährung',
    'Notierungsaufnahme'
]
df = df.drop_duplicates(subset=subset_cols, keep="first")

# 5. Create emission year fixed effects from "Emissionsdatum"
if "Emissionsdatum" in df.columns:
    df["Emissionsdatum"] = pd.to_datetime(df["Emissionsdatum"], format="%d,%m,%Y", errors="coerce")
    df["Emission_year"] = df["Emissionsdatum"].dt.year
    # Create dummy variables for emission year (fixed effects)
    df = pd.get_dummies(df, columns=["Emission_year"], prefix="Emission_year", drop_first=False)

# (Optional) Display the first few rows to verify the changes
print(df.head())
df.to_csv("bond_universe_cleaned.csv", index=False)

                                                 URL          ISIN  \
0  https://www.boerse-frankfurt.de/anleihe/xs2694...  xs2694874533   
2  https://www.boerse-frankfurt.de/anleihe/xs2482...  xs2482887879   
3  https://www.boerse-frankfurt.de/anleihe/xs2482...  xs2482887879   
4  https://www.boerse-frankfurt.de/anleihe/de000a...  de000a3lh6t7   
5  https://www.boerse-frankfurt.de/anleihe/de000a...  de000a3lh6t7   

                                   Company Kupon_Maturity  Coupon  \
0                  volkswagen-leasing-gmbh     4-75-23-31    4.75   
2                                   rwe-ag     2-75-22-30    2.75   
3                                   rwe-ag     2-75-22-30    2.75   
4  mercedes-benz-international-finance-b-v      3-5-23-26    3.50   
5  mercedes-benz-international-finance-b-v      3-5-23-26    3.50   

   Maturity_Start  Maturity_End  is_green Letzter Preis  \
0              23            31         1        106,88   
2              22            30         1     